# Simulate experiments

In [1]:
import pandas as pd
import numpy as np
from ipywidgets import widgets, interact
from stockmarket import baselinemodel
import sqlite3
#from tqdm import *
import json
import os
import shutil

## Create a base json file with experiment properties

In [2]:
parameters = {
    'seed' : 0, 
    'simulation_time' : 100, 
    'amount_fundamentalists' : 40, 
    'amount_chartists' : 40, 
    'amount_firms': 4,
    'initial_money' : (100, 200),
    'initial_bid_ask' : (5, 5),
    'initial_memory' : (1, 4),
    'initial_ma_short' : (1, 3),
    'initial_ma_long' : (6, 7),
    'initial_profit' : (200, 200),
    'initial_book_value' : (10000, 10000),
    'initial_stock_amount' : 400,
    'observable_set_size' : 3
}

## Create a dictionary with all experiments nescessary for chartist v fundamentalists

In [3]:
LOW = 0
STEPS = 5
HIGH = 100

experiments = {}
for i in range(int((HIGH - LOW) / STEPS)):
    experiments['experiment{}'.format(i+1)] = (LOW + STEPS*(i)) / HIGH

In [4]:
AGENTS = 50
NRUNS = 1

## Create experiments folders

In [5]:
parent_dir = os.getcwd()
# if there is a data folder, delete it and its content
try: 
    shutil.rmtree('data')
except:
    pass

In [6]:
# make a data folder and go into it
dataFolder = 'data'
os.makedirs(dataFolder)
os.chdir(dataFolder)
for key in experiments:
    # add a folder for every experiment 
    foldername = key
    os.makedirs(foldername)
    # create experiments parameter set
    experimentParams = parameters.copy()
    experimentParams['amount_chartists'] = int(AGENTS * experiments[key])
    experimentParams['amount_fundamentalists'] = int(AGENTS * (1 - experiments[key]))
    # create json experiments file
    with open('{}/parameters.json'.format(foldername), 'w') as fp:
        json.dump(experimentParams, fp)

# Run experiments for every folder

In [7]:
for key in experiments:
    os.chdir(key)
    parameters = json.load(open('parameters.json'))
    conn = sqlite3.connect('dataBaseOffline.sqlite')
    cur = conn.cursor()
    for seed in tqdm(range(NRUNS)):
        Transactions, Transactors, Statevariables, Variabletypes, Objects = baselinemodel.stockMarketSimulation(
        seed=parameters['seed'],
        simulation_time=parameters['simulation_time'],
        amount_fundamentalists=parameters['amount_fundamentalists'],
        amount_chartists=parameters['amount_chartists'], 
        amount_firms=parameters['amount_firms'],
        initial_money=parameters['initial_money'], 
        initial_bid_ask=parameters['initial_bid_ask'],
        initial_memory=parameters['initial_money'],
        initial_ma_short=parameters['initial_ma_short'], 
        initial_ma_long=parameters['initial_ma_long'], 
        initial_profit=parameters['initial_profit'],
        initial_book_value=parameters['initial_book_value'], 
        initial_stock_amount=parameters['initial_stock_amount'], 
        observable_set_size=parameters['observable_set_size']
        )
        Transactions.to_sql("Transactions{}".format(seed), conn, if_exists="replace")
        Transactors.to_sql("Transactors{}".format(seed), conn, if_exists="replace")
        Statevariables.to_sql("Statevariables{}".format(seed), conn, if_exists="replace")
        Variabletypes.to_sql("Variabletypes{}".format(seed), conn, if_exists="replace")
        Objects.to_sql("Objects{}".format(seed), conn, if_exists="replace")
    conn.commit()
    cur.close()
    os.chdir('{}/{}'.format(parent_dir,dataFolder))